# Satellite data: channel VIS06 (visible  0.635 µm)

The aim of this notebook is to show how to read the visible channel satellite data from a data sample. 

You will find one NetCDF file per geographic zone ('NW' for North-West of France and 'SE' for South-East of France) and per year. 

The satellites emit signals at different wavelengths and capture signals reflected and/or re-emitted by the clouds or by the earth surface in clear skies. A channel means a range of wavelenghts : visible, infrared... 

About visible channel, for each pixel, the value represents the radiance (in %), which represents the proportion of radiation reflected by the target (clouds or ground, sea if there are no cloud), in the visible domain. 

The spatial resolution is 0.03° and the time step is 1 hour but some dates can be missing (due to problems in original data, so we can do anything). The missing values are -1.

/!\ The visible channel is by definition not visible at night; there are huge gaps of distribution between the day and night.

For more information, cf [documentation](https://meteofrance.github.io/meteonet/english/data/satellite/#21-visible-channel-vis).

# Note

<font size="4.5">To use <span style="color:blue">**Cartopy**</span>, a library to plot data with basemaps (see cells below), it is necessary to <span style="color:red">activate the internet connection</span> of that notebook (in edit mode, you can find on the right column, in the *Settings* section, a row entitled *Internet*, put the slider bar on **on**).  </font>

In [ ]:
import xarray as xr 
from matplotlib import colors
import matplotlib.pyplot as plt
import datetime
import json

## Parameters definition

Select the area and year you want to study:

In [ ]:
zone = 'NW'    #on Kaggle, only the North-West zone is available
year = '2017'

## Loading data using parameters

In [ ]:
channel = 'VIS06'   #channel name
fname = "/kaggle/input/meteonet/Satellite_"+ channel + "/Satellite_" + channel + "/" + channel + "_%s_%s.nc" % (zone,year) #it is about a data sample : the first day in 2016 in SE zone
data = xr.open_dataset(fname)

## Quick field visualisation 

The following command shows the satellite data on a given zone at different dates (dates indexes : 11,12,14 and 15).

In [ ]:
#get the color palette information (a black and white color palette from the intern forecasting tool)

#color palette about VIS06 channel satellite data
def vis_palette(palette_name):
    m = []
    M = []
    c = []
    with open('/kaggle/input/meteonet/color_palettes/color_palettes/'+ palette_name) as f:
        color_data = json.load(f)
    for i in range(0,len(color_data['levels'])):
        m.append(color_data['levels'][i]['min'])
        M.append(color_data['levels'][i]['max'])
        c.append(color_data['levels'][i]['color'])
    m.append(M[len(M)-1])
    m.insert(0,-1)
    c.insert(0,'dodgerblue')
    return m,c

palette_name = "VIS_palette.json"  
bounds, c = vis_palette(palette_name)  
cmap = colors.ListedColormap(c)
norm = colors.BoundaryNorm(bounds, cmap.N)

In [ ]:
#a black and white color palette from the intern forecasting tool : could be not clear enough, according to dates 
data.isel(time=[11,12,14,15])[channel].plot(x='lon', y='lat',col='time',cmap=cmap, norm=norm) 
#without a defined color palette 
data.isel(time=[11,12,14,15])[channel].plot(x='lon', y='lat',col='time')
plt.show()

## Exploration of metadata

Overview of the data:

In [ ]:
data

Get the information about coordinates (latitude and longitude). It is in degrees (north degrees about latitude and east degrees about longitude).

In [ ]:
coord = 'lon'
data[coord]

Get the information about the observation times:

In [ ]:
dates = data['time']
dates

Get the information about the visible satellite data:

In [ ]:
d = data[channel]
d_vals = d.values  #get the values 
d[0:2,0:2,0:2]

The structure of the data (3 dimensions):
* number of observation times
* number of points in latitude 
* number of points in longitude

In [ ]:
d_vals.shape

## Plotting data with Cartopy

Plot the data for one given date with Cartopy:

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
#studied date
studied_date = '201701011200'
date = datetime.datetime.strptime(studied_date,'%Y%m%d%H%M')

color_palette = False  #if False, then no defined color palette 

In [ ]:
#Coordinates of studied area boundaries (in °N and °E)
lllat = 46.25  #lower left latitude
urlat = 51.896  #upper right latitude
lllon = -5.842  #lower left longitude
urlon = 2  #upper right longitude
extent = [lllon, urlon, lllat, urlat]
fig = plt.figure(figsize=(8,9))

# Select projection
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot the data
if color_palette == True:
    img = plt.imshow(data.sel(time=date)[channel], interpolation='none', origin='upper',cmap=cmap, norm=norm, extent=extent)
else:
    img = plt.imshow(data.sel(time=date)[channel], interpolation='none', origin='upper', extent=extent)
# Add coastlines and borders
ax.coastlines(resolution='50m', linewidth=1)
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

# Show only the area we defined
ax.set_extent(extent)

plt.colorbar(img, orientation= 'horizontal', pad=0.03).set_label(data[channel].Description + ' / Missing value : -1')
plt.title(channel + ' data - '+ zone + ' zone - '+ studied_date)
plt.show()